In [1]:
# http://www.pianshen.com/article/66383398/  论文及翻译
import tensorflow as tf
import os                            # 实现操作系统的许多功能 用于文件管理
import tarfile                       # 文件打包压缩解包解压缩
import requests

In [2]:
# inception模型下载地址 这个inception-v3模型的参数已经训练好了 下载来用就行
inception_pretrain_model_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'

# 模型存放地址 如果这个文件夹不存在 就先创建这个文件夹
inception_pretrain_model_dir = 'inception_model'
if not os.path.exists(inception_pretrain_model_dir):
    os.makedirs(inception_pretrain_model_dir)
    
# 获取文件名以及文件路径
# 拆分字符串：str.split(str='',num=string.count(str))[n] 
# 字符串str、以''中的符号为分隔符、分割num次、取第[n]个分片 [-1]最后一个就是 inception-2015-12-05.tgz
filename = inception_pretrain_model_url.split('/')[-1]
# os.path.join()路径拼接文件路径
filepath = os.path.join(inception_pretrain_model_dir,filename)# =inception_model\\inception-2015-12-05.tgz

#下载模型
if os.path.exists(filepath):
    print('已下载: ',filename)
else:
    print('下载: ',filename)
# 默认情况下，当进行网络请求后，响应体会立即被下载。可以通过 stream 参数覆盖这个行为，推迟下载响应体直到访问 Response.content 属性
# 此时仅有响应头被下载下来了，连接保持打开状态，因此允许我们根据条件获取内容
# http://www.cnblogs.com/nul1/p/9172068.html 这个文档讲的很清楚 这一段下载就是为了防止大文件占用内存 这样子分块下载就OK了
    r = requests.get(inception_pretrain_model_url,stream = True)
    with open(filepath,'wb') as f:
        for chunk in r.iter_content(chunk_size = 1024):
            if chunk:
                f.write(chunk)
    print('finish: ',filename)

# 解压文件  https://cloud.tencent.com/developer/section/1366828
tarfile.open(filepath,'r:gz').extractall(inception_pretrain_model_dir) # 将存档中的所有成员提取到当前工作目录或目录路径
print('解压完成')

# 等会儿模型结构的存放文件
log_dir = 'inception_log'
print('等会儿模型结构存放在: ',log_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# classify_image_graph_def.pb为google训练好的模型 = inception_graph_def_file
inception_graph_def_file = os.path.join(inception_pretrain_model_dir,'classify_image_graph_def.pb')

with tf.Session() as sess:
    # 从pb文件中调用   https://blog.csdn.net/zj360202/article/details/78539464
    with tf.gfile.FastGFile(inception_graph_def_file, 'rb') as f:
    # tf.gfile.FastGFile(path,decodestyle) 
    # 函数功能：实现对图片的读取。 
    # 函数参数：(1)path：图片所在路径 (2)decodestyle:图片的解码方式。(‘r’:UTF-8编码; ‘rb’:非UTF-8编码)
        
        # 声明一个默认的图，然后定义张量内容，在后面可以调用或保存
        graph_def = tf.GraphDef()
        # tf.Graph()可以1.通过tensorboard用图形化界面展示出来流程结构 2.可以整合一段代码为一个整体存在于一个图中

        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def,name = '')
    # 保存图的结构 路径是log_dir = 'inception_log'
    writer = tf.summary.FileWriter(log_dir,sess.graph)
    writer.close()


已下载:  inception-2015-12-05.tgz
解压完成
等会儿模型结构存放在:  inception_log


In [3]:
# 用tensorboard打开路径log_dir = 'inception_log'下的图：
# tensorboard --logdir=G:\Anaconda3\My_Jupyter\inception_log